## HQC

In [579]:
%run ProductCode.ipynb
%run ConcatenatedCode.ipynb
%run Conversions.ipynb

Codeword:  100101000111111100011111010011000001111111110110010110111001010110001010010001011010011001101110101110100010111110001000010010100010111011000100111101011011011001110011110100011100111000000001111001010110010101001001100110110011001010110000101100110001111101011000111111001011000101101011001101001001110000110000011100011011011101101010110010010001101010010100011110011100100110101110111010000100011001100101111111011110
111011110101000111011110101000111011110101000111011110101000111011110101000111011110
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True
Decoding status:  True


IndexError: list index out of range

IndexError: list index out of range

In [559]:
class HQC:
    
    def __init__(self, w, w_e, w_r, C, key_type = 'pol'):
        
        self.w = w
        self.w_e = w_e
        self.w_r = w_r
        self.C = C
        self.q = 2
        
        if (self.C.C1.q == self.C.C2.q and self.C.C1.q != self.q):
            self.n = self.C.n * self.C.C1.power
            self.k = self.C.k * self.C.C1.power
        else:
            self.n = self.C.n
            self.k = self.C.k
            
        
        self.F = GF(self.q)
        self.R = PolynomialRing(self.F, 'x'); x = self.R.gen()
        self.S = QuotientRing(self.R, x**self.n - 1, 'a'); a = self.S.gen()
        
        self.pk, self.sk = self.KeyGen(out = key_type)
        
        
    def KeyGen(self, out = 'pol'):
        
        h = self.R.random_element(degree=(0,self.n))
        h = self.S(h)
        
        x = [0] * self.n
        x = self.RandomGenWithWeight(x, self.w)
        x = self.S(x)
        
        y = [0] * self.n
        y = self.RandomGenWithWeight(y, self.w)
        y = self.S(y)
        
        s = x + h * y
        
        if out == 'pol':
            pk = (h, s)
            sk = (x, y)
        elif out == 'int':
            pk = (_PolToInt(h, self.q), _PolToInt(s, self.q))
            sk = (_PolToInt(x, self.q), _PolToInt(y, self.q))
        elif out == 'bin':
            pk = (_PolToInt(h, self.q), _PolToInt(s, self.q))
            sk = (_PolToInt(x, self.q), _PolToInt(y, self.q))
            pk = (_IntToBitString(pk[0], self.q), _IntToBitString(pk[1], self.q))
            sk = (_IntToBitString(sk[0], self.q), _IntToBitString(sk[1], self.q))
        
        return pk, sk
    
    
    def Encrypt(self, message, zeropad = True, out = 'bin'):
        
        pk_data_type = _DetermineInput(self.pk[0], self.q)
        
        if pk_data_type == 'pol':
            pk = self.pk
        elif pk_data_type == 'int':
            pk = (self.S(self.pk[0]), self.S(self.pk[1]))
        elif pk_data_type == 'bin':
            pk = (_BitStringToInt(self.pk[0], self.q), _BitStringToInt(self.pk[1], self.q))
            pk = (self.S(pk[0]), self.S(pk[1]))
        
        
        c = self.C.Encoding(message, out = 'bin')
        c = _BitStringToInt(c, self.q)
        
        u = []
        v = []
        
        for i in range(0, len(c), self.n):
            e = [0] * self.n
            e = self.RandomGenWithWeight(e, self.w_e)
            e = self.S(e)

            r1 = [0] * self.n
            r1 = self.RandomGenWithWeight(r1, self.w_r)
            print(r1)
            r1 = self.S(r1)
            

            r2 = [0] * self.n
            r2 = self.RandomGenWithWeight(r2, self.w_r)
            print(r2)
            r2 = self.S(r2)
            
            print('here', _PolToInt(r1 + pk[0] * r2, self.q))
            u.extend(r1 + pk[0] * r2)
            v.extend(self.S(c[i:i+self.n]) + pk[1] * r2 + e)
            
        if out == 'pol':
            u, v = (self.S(u), self.S(v))
        elif out == 'int':
            u, v = (_PolToInt(u, self.q), _PolToInt(v, self.q))
        elif out == 'bin':
            u, v = (_PolToInt(u, self.q), _PolToInt(v, self.q))
            u, v = (_IntToBitString(u, self.q), _IntToBitString(v, self.q))
        else:
            raise ValueError('Unrecognized data type')
            
        return (u, v)
            
    
    def Decrypt(self, c, out = 'bin'):
        
        sk_data_type = _DetermineInput(self.sk[0], self.q)
        
        if sk_data_type == 'pol':
            sk = self.sk
        elif sk_data_type == 'int':
            sk = (self.S(self.sk[0]), self.S(self.sk[1]))
        elif sk_data_type == 'bin':
            sk = (_BitStringToInt(self.sk[0], self.q), _BitStringToInt(self.sk[1], self.q))
            sk = (self.S(sk[0]), self.S(sk[1]))
            
            
        c_data_type = _DetermineInput(c[0], self.q)
        
        if c_data_type == 'int':
            pass
        elif c_data_type == 'bin':
            c = (_BitStringToInt(c[0], self.q), _BitStringToInt(c[1], self.q))
        
        
        d_out = []
        
        for i in range(0, len(c[0]), self.n):
            d = self.S(c[1][i:i+self.n]) - self.S(c[0][i:i+self.n]) * sk[1]
            d = _PolToInt(d, self.q)
            d = _IntToBitString(d, self.q)
            d = self.C.Decoding(d, out = 'bin')
            d = _BitStringToInt(d, self.q)
            d_out.extend(d)
        
        
        if out == 'int':
            pass
        elif out == 'bin':
            d_out = _IntToBitString(d_out, self.q)
        else:
            raise ValueError('Unrecognized data type')
        
        return d_out
        
        
    def RandomGenWithWeight(self, array, weight):

        for i in range(weight):
            cur = ZZ.random_element(0,len(array))
            while array[cur] == 1:
                cur = ZZ.random_element(0,len(array))
            
            array[cur] = 1
        
        return array

In [576]:
#C = HQC(w = 5, w_e = 5, w_r = 5, C = ProductCode(BCHCode(n = 15, b = 1, D = 7), RepetitionCode(n = 31)), key_type = 'bin')
C = HQC(w = 7, w_e = 5, w_r = 5, C = ProductCode(RSCode(n=15, k=7, q=2**4), RSCode(n=15, k=11, q=2**4)), key_type = 'int')
#C = HQC(w = 10, w_e = 10, w_r = 10, C = ProductCode(RMCode(r = 1, m = 7), RMCode(r = 1, m = 7)), key_type = 'int')

#C = HQC(w = 60, w_e = 60, w_r = 60, C = ConcatenatedCode(RSCode(n = 255, k = 225, q = 2**8), RMCode(r = 1, m = 7)), key_type = 'bin')


print('Public key: ', C.pk)
print('\nSecret key: ', C.sk)

Public key:  ([0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0

In [577]:
m = '10101'

c = C.Encrypt(m, out = 'bin')


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [578]:
d = C.Decrypt(c, out = 'bin')
print(d)

00001001110011101001111100100010000011000110000100000000001000001100011100010000000010000000110001100001000001000010000011000110010100000100001000001100011000010000000100100000110001100011000010000010000010000110000100000000001000001101011000010000000000100000110001010001000000000010000011000110000100000001
